In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/burns-trauma.csv")

In [3]:
cvp=pd.read_csv("CVP.csv")

In [4]:
burns_merged = pd.merge(cvp[['patientunitstayid','val']], burns, how='right', on='patientunitstayid')

In [5]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [6]:
burns_merged_2 = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns_merged, how='right', on='patientunitstayid')

In [7]:
burns_merged_2['mods_ratio'] = 0

In [8]:
burns_merged_2

,patientunitstayid,pao2,fio2,val,gender,age,ethnicity,apacheadmissiondx,admissionheight,hospitaladmitoffset,...,Hgb,WBC x 1000,platelets x 1000,RBC,bicarbonate,MCV,MCHC,MCH,RDW,mods_ratio
0,143103,346.0,100.0,5.333333,Female,46,Asian,Head only trauma,167.6,-3.0,...,12.10,19.50,258.0,3.98,20.0,85.9,35.4,30.4,13.50,0
1,143258,-1.0,-1.0,NaN,Male,90,Caucasian,"Rhythm disturbance (atrial, supraventricular)",180.3,0.0,...,11.90,8.60,175.0,3.48,28.0,102.0,33.5,34.2,12.10,0
2,143274,-1.0,-1.0,NaN,Male,22,Caucasian,Head/extremity trauma,195.6,-179.0,...,12.10,14.50,218.0,4.44,30.0,84.0,32.4,27.3,12.70,0
3,145180,-1.0,-1.0,NaN,Female,77,-,"Extremity only trauma, surgery for",167.6,-1891.0,...,10.00,5.40,173.0,3.29,25.0,94.2,32.3,30.4,13.70,0
4,146784,80.0,40.0,NaN,Male,50,Caucasian,Spinal/face trauma,177.8,-2.0,...,11.50,14.00,164.0,3.66,25.0,93.4,33.6,31.4,12.90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,3352198,-1.0,-1.0,NaN,Male,61,Other/Unknown,Chest/face trauma,188.0,-276.0,...,14.80,13.80,256.0,4.95,27.5,89.0,34.0,30.0,13.60,0
4911,3352519,-1.0,-1.0,NaN,Male,46,Caucasian,Head/multiple trauma,172.7,-232.0,...,13.20,12.40,178.0,4.47,27.0,87.0,34.0,30.0,13.30,0
4912,3352711,112.0,100.0,NaN,Male,53,Caucasian,"Sepsis, pulmonary",172.7,-13.0,...,10.70,10.80,272.0,3.39,21.0,96.0,33.0,32.0,13.60,0
4913,3353007,-1.0,-1.0,NaN,Female,74,Caucasian,Chest/extremity trauma,157.5,-66.0,...,11.55,11.25,197.0,3.77,29.5,94.0,33.0,30.5,12.75,0


In [9]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2.iloc[i]['pao2'] != -1) & (burns_merged_2.iloc[i]['fio2'] != -1):
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 300) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 226):
            burns_merged_2['mods_ratio'].iloc[i] = 1
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 225) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 151):
            burns_merged_2['mods_ratio'].iloc[i] = 2
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 150) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 76):
            burns_merged_2['mods_ratio'].iloc[i] = 3
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 0) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 75):
            burns_merged_2['mods_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
def platelets(item):
    temp = 0
    if (item >= 81) & (item <= 120):
        temp += 1
    elif (51 <= item) & (item <= 80):
        temp += 2
    elif (21 <= item) & (item <= 50):
        temp += 3
    elif (20 >= item):
        temp += 4
    return temp

In [11]:
burns_merged_2['mods_platelets'] = burns_merged_2['platelets x 1000'].apply(platelets)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_3 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged_2, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.3) & (item <= 3.5):
        temp += 1
    elif (3.6 <= item) & (item <= 7.0):
        temp += 2
    elif (7.0 <= item) & (item <= 14.0):
        temp += 3
    elif (14.0 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_3['mods_bilirubin'] = burns_merged_3['bilirubin'].apply(bilirubin)

In [16]:
burns_merged_3['val_fill'] = burns_merged_3['val'].fillna(burns_merged_3['val'].mean())

In [17]:
burns_merged_3['PAHR'] = (burns_merged_3['heartrate'] * burns_merged_3['val'])/burns_merged_3['meanbp']

In [18]:
burns_merged_3['PAHRF'] = (burns_merged_3['heartrate'] * burns_merged_3['val_fill'])/burns_merged_3['meanbp']

In [19]:
burns_merged_3['PAHRF'].mean()

21.91657767132443

In [20]:
def ad_pressure(item):
    temp = 0
    if (item >= 10.1) & (item <= 15):
        temp += 1
    elif (item >= 15.1) & (item <= 20):
        temp += 2
    elif (item >= 20.1) & (item <= 30):
        temp += 3
    elif (item >= 30.1):
        temp += 4
    return temp

In [21]:
burns_merged_3['mods_pressure'] = burns_merged_3['PAHR'].apply(ad_pressure)

In [22]:
burns_merged_3['GCS'] = burns_merged_3['eyes'] + burns_merged_3['motor'] + burns_merged_3['verbal']

In [23]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [24]:
burns_merged_3['mods_GCS'] = burns_merged_3['GCS'].apply(GCS)

In [25]:
def creatinine(item):
    temp = 0
    if (item >= 1.2) & (item <= 2.2):
        temp += 1
    elif (2.3 <= item) & (item <= 3.9):
        temp += 2
    elif (4 <= item) & (item <= 5.7):
        temp += 3
    elif (5.7 < item):
        temp += 4
    return temp

In [26]:
burns_merged_3['mods_creatinine'] = burns_merged_3['creatinine'].apply(creatinine)

In [27]:
burns_merged_3.columns

Index(['patientunitstayid', 'bilirubin', 'pao2', 'fio2', 'val', 'gender',
       'age', 'ethnicity', 'apacheadmissiondx', 'admissionheight',
       'hospitaladmitoffset', 'unittype', 'unitadmitsource', 'unitstaytype',
       'admissionweight', 'intubated', 'dialysis', 'temperature',
       'respiratoryrate', 'heartrate', 'meanbp', 'hematocrit',
       'actualicumortality', 'preopmi', 'preopcardiaccath', 'ptcawithin24h',
       'meds', 'verbal', 'motor', 'eyes', 'thrombolytics', 'aids',
       'hepaticfailure', 'lymphoma', 'immunosuppression', 'cirrhosis',
       'activetx', 'ima', 'midur', 'ventday1', 'oobventday1', 'oobintubday1',
       'diabetes', 'diagnosisstring', 'potassium', 'creatinine', 'sodium',
       'BUN', 'glucose', 'chloride', 'calcium', 'Hgb', 'WBC x 1000',
       'platelets x 1000', 'RBC', 'bicarbonate', 'MCV', 'MCHC', 'MCH', 'RDW',
       'mods_ratio', 'mods_platelets', 'mods_bilirubin', 'val_fill', 'PAHR',
       'PAHRF', 'mods_pressure', 'GCS', 'mods_GCS', 'mods_cre

In [28]:
burns_merged_3['mods_score'] = burns_merged_3[['mods_ratio', 'mods_platelets', 'mods_bilirubin',
        'mods_pressure', 'mods_GCS', 'mods_creatinine']].sum(axis=1)

In [29]:
burns_merged_3['actualicumortality_label'] = burns_merged_3['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [30]:
def mods_predict(item):
    temp = 0
    if (item >= 1) & (item <= 4):
        temp = 1.5
    elif (item >= 5) & (item <= 8):
        temp = 4.0
    elif (item >= 9) & (item <= 12):
        temp = 25
    elif (item >= 13) & (item <= 16):
        temp = 50
    elif (item >= 17) & (item <= 20):
        temp = 75
    elif (item >= 21) & (item <= 24):
        temp = 100
    return temp

In [31]:
burns_merged_3['mods_score'].max()

19

In [32]:
burns_merged_3['mods_prob'] = burns_merged_3['mods_score'].apply(mods_predict)

In [33]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_3['actualicumortality_label'].values, burns_merged_3['mods_prob'].values, pos_label=1)

In [34]:
metrics.auc(fpr, tpr)

0.8024665488513001

In [35]:
burns_merged_3['mods_prob']

0       1.5
1       1.5
2       1.5
3       1.5
4       1.5
       ... 
4910    0.0
4911    1.5
4912    4.0
4913    0.0
4914    0.0
Name: mods_prob, Length: 4915, dtype: float64